In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import sys
sys.path.append('/Users/keenanfryer/projects/thesis-plotting/src')

In [3]:
import grib_reader_ensemble
from iris_grib_mappings import _field_keyval_mapping
from glob import glob
import xarray
import numpy
import seaborn as sns
import pandas
from read_sounding_observation import read_fixed_width_sounding
import os
from src.ensemble_datasets import Dataset

In [4]:
souding_location_meta = {
    "OKX":{
        "full_name":"Upton, NY",
        "station_id": 72501,
        "lat":40.87,
        "lon":-72.86
    },
    "CHH":{
        "full_name":"Upton, NY",
        "station_id": 74494,
        "lat":41.66,
        "lon":-69.96,
    },
}

In [4]:
single_member_files = glob('/Volumes/Seagate/thesis_project/thesis_project_data/grib_output_files/YesradRun/wrfout_201302*0001.grib2')
single_member_files.sort()
tind = 4

In [5]:
field_list = _field_keyval_mapping.keys()
field_list = [f for f in field_list if "surface" in f]

In [6]:
single_member_files.

[]

In [7]:
single_member_files[tind].replace("0001.grib2","000*.grib2").replace("YesradRun","NoradRun")

'/Volumes/Seagate/thesis_project/thesis_project_data/grib_output_files/NoradRun/wrfout_20130209T010000_000*.grib2'

In [15]:
cubes = grib_reader_ensemble.load_iris_cubes(single_member_files[tind].replace("0001.grib2","0002.grib2"),field_list=field_list)
data_array_list = [xarray.DataArray.from_iris(c) for c in cubes]
isobar,surface,coord = grib_reader_ensemble._split_data_array_by_level_type(data_array_list)

In [23]:
da = surface[0]

In [33]:
a = da.assign_coords({'time':da['time'],'forecast_reference_time':da['forecast_reference_time']})

In [34]:
a.expand_dims(['time','forecast_reference_time'])

<xarray.Dataset>
Dimensions:                  (projection_y_coordinate: 183,
                              projection_x_coordinate: 189, time: 1,
                              forecast_reference_time: 1)
Coordinates:
  * projection_y_coordinate  (projection_y_coordinate) float64 9.489e+05 ... ...
  * projection_x_coordinate  (projection_x_coordinate) float64 3.211e+05 ... ...
  * forecast_reference_time  (forecast_reference_time) datetime64[ns] 2013-02-09
  * time                     (time) datetime64[ns] 2013-02-09T01:00:00
Data variables:
    forecast_period          (time, forecast_reference_time) timedelta64[ns] ...
    mslp_surface             (time, forecast_reference_time, projection_y_coordinate, projection_x_coordinate) float64 dask.array<chunksize=(1, 1, 183, 189), meta=np.ndarray>

In [22]:
out_arrays = []
for da in surface:
    da.drop_vars()
    out_arrays.append()

<xarray.DataArray 'time' ()>
[1 values with dtype=datetime64[ns]]
Attributes:
    standard_name:  time
<xarray.DataArray 'time' ()>
[1 values with dtype=datetime64[ns]]
Attributes:
    standard_name:  time
<xarray.DataArray 'time' ()>
[1 values with dtype=datetime64[ns]]
Attributes:
    standard_name:  time
<xarray.DataArray 'time' ()>
[1 values with dtype=datetime64[ns]]
Attributes:
    standard_name:  time
<xarray.DataArray 'time' ()>
[1 values with dtype=datetime64[ns]]
Attributes:
    standard_name:  time
<xarray.DataArray 'time' ()>
[1 values with dtype=datetime64[ns]]
Attributes:
    standard_name:  time
<xarray.DataArray 'time' ()>
[1 values with dtype=datetime64[ns]]
Attributes:
    standard_name:  time
<xarray.DataArray 'time' ()>
[1 values with dtype=datetime64[ns]]
Attributes:
    standard_name:  time


In [21]:
surface[0].drop_vars(['forecast_period','height'],errors="ignore").assign_coords({
    'time':surface[0]['time'],
    'forecast_reference_time':surface[0]['forecast_reference_time'],
    })

<xarray.Dataset>
Dimensions:                  (projection_y_coordinate: 183,
                              projection_x_coordinate: 189)
Coordinates:
  * projection_y_coordinate  (projection_y_coordinate) float64 9.489e+05 ... ...
  * projection_x_coordinate  (projection_x_coordinate) float64 3.211e+05 ... ...
    forecast_reference_time  datetime64[ns] ...
    time                     datetime64[ns] ...
Data variables:
    mslp_surface             (projection_y_coordinate, projection_x_coordinate) float64 dask.array<chunksize=(183, 189), meta=np.ndarray>

In [8]:
yes_ds = Dataset(single_member_files[tind].replace("0001.grib2","00[0-1]*.grib2"),ensemble_data=True)
no_ds = Dataset(single_member_files[tind].replace("0001.grib2","00[0-1]*.grib2").replace("YesradRun","NoradRun"),ensemble_data=True)

Frozen({'projection_y_coordinate': 183, 'projection_x_coordinate': 189, 'pressure': 46, 'member': 19})
Frozen({'projection_y_coordinate': 183, 'projection_x_coordinate': 189, 'member': 19})
Frozen({'projection_y_coordinate': 183, 'projection_x_coordinate': 189, 'member': 19})


ValueError: 'time' is not in list

In [ ]:
yes_ds.isobaric_data

In [ ]:
def prep_profile_for_plotting(profile_dataset,time_index):
    
    profile_time = profile_dataset.isel(time=time_index).to_dataframe()
    #profile_time = profile_time.sort_values('pressure',ascending=False)
    profile_time['temperature_C'] = profile_time['temperature'] - 273.15
    return profile_time

In [ ]:
station = "OKX"
yesprofile = yes_ds.get_isobaric_profile_dataset(numpy.array(souding_location_meta[station]['lat']),numpy.array(souding_location_meta[station]['lon']))
noprofile = no_ds.get_isobaric_profile_dataset(numpy.array(souding_location_meta[station]['lat']),numpy.array(souding_location_meta[station]['lon']))


In [ ]:
observed_profile = read_fixed_width_sounding(os.path.join('/Users/keenanfryer/projects/thesis-plotting/sounding_observations',f"{souding_location_meta[station]['station_id']}_20130208_1200Z"))

In [ ]:
observed_profile

In [ ]:
yes_plot_profile = yesprofile.to_dataframe()
no_plot_profile = noprofile.to_dataframe()

In [ ]:
yes_plot_profile['temperature_C'] = yes_plot_profile['temperature'] - 273.15
no_plot_profile['temperature_C'] = no_plot_profile['temperature'] - 273.15

In [ ]:
def subset_profile_by_pressure(indf,pressure_ceiling,pressure_keyname='pressure'):
    return indf.query(f'{pressure_keyname} >= @pressure_ceiling').copy()

In [ ]:
yes_plot_profile = subset_profile_by_pressure(yes_plot_profile,500)
no_plot_profile = subset_profile_by_pressure(no_plot_profile,500)

In [ ]:
valid_time = pandas.to_datetime(yes_plot_profile.time.unique()[0])

In [ ]:
ax = sns.lineplot(data=subset_profile_by_pressure(yes_plot_profile,500),x='temperature_C',y='pressure',orient='y',color='red',**{'label':'yes-radar'})
ax = sns.lineplot(data=subset_profile_by_pressure(no_plot_profile,500),x='temperature_C',y='pressure',orient='y',color='blue',**{'label':'no-radar'})
ax = sns.lineplot(data=subset_profile_by_pressure(observed_profile,500,pressure_keyname='pressure_hPa'),x='temperature_C',y='pressure_hPa',orient='y',**{'color':'black'})
ax.invert_yaxis()
ax.figure.savefig(f"Soudnding_comparison_{station}_{valid_time:%Y%m%dT%H%M}")
